![](https://www.txstate.edu/cache78a0c25d34508c9d84822109499dee61/imagehandler/scaler/gato-docs.its.txstate.edu/jcr:21b3e33f-31c9-4273-aeb0-5b5886f8bcc4/fake-fact.jpg?mode=fit&width=1600)

## We will be using RoBERTa algorithm from Hugging Face library of transformers for NLP. Transformers are cutting edge technology now in NLP.

In [ ]:
import pandas as pd
import numpy as np

true, fake = pd.read_csv("../input/fake-and-real-news-dataset/True.csv"), pd.read_csv("../input/fake-and-real-news-dataset/Fake.csv")
true.head()

In [ ]:
fake['target'] = 0 
true['target'] = 1

In [ ]:
fake

In [ ]:
news = pd.concat([true,fake],axis=0,ignore_index=True)
news = news.sample(frac = 1).reset_index(drop = True)
news.head()

# Defining the tokenizer function

In [ ]:
import tensorflow as tf
import transformers
import tqdm
from keras.preprocessing import sequence

#creating a function
def func_tokenizer(tokenizer_name, docs):
    features = []
    for doc in tqdm.tqdm(docs, desc = 'converting documents to features'):
        tokens = tokenizer_name.tokenize(doc)
        ids = tokenizer_name.convert_tokens_to_ids(tokens)
        features.append(ids)
    return features
print("The function is created successfully")

In [ ]:
from sklearn.model_selection import train_test_split
X, y = news['text'], news['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# All hail Roberta from hugging face library of transformers 🤗

In [ ]:
#Initialize bert tokenizer
roberta_tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base-openai-detector')

In [ ]:
roberta_train_features = func_tokenizer(roberta_tokenizer, X_train)
roberta_test_features = func_tokenizer(roberta_tokenizer, X_test)

In [ ]:
roberta_trg = sequence.pad_sequences(roberta_train_features, maxlen = 500)
roberta_test = sequence.pad_sequences(roberta_test_features, maxlen = 500)

# XGBoost

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 1000, learning_rate = 0.15, max_depth = 9,
                    eval_metric = 'auc', use_label_encoder=False,objective = 'binary:logistic')
xgb.fit(roberta_trg, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

xgb_pred = xgb.predict(roberta_test)
xgb_score = accuracy_score(y_test, xgb_pred)
xgb_roc = roc_auc_score(y_test, xgb_pred)
print("The accuracy of XGBOOST is: %0.2f" %xgb_score)
print("The roc_auc score of XGBOOST is: %0.2f" %xgb_roc)

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(y_test, xgb_pred)
print(cr)

# CatBoost

In [ ]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier(eval_metric = 'Accuracy', iterations = 2000, learning_rate = 0.2)

In [ ]:
cb.fit(roberta_trg, y_train, verbose = 0)
cb_pred = cb.predict(roberta_test)
cb_score = accuracy_score(y_test, cb_pred)
cb_roc = roc_auc_score(y_test, cb_pred)
print("The accuracy of CatBoost is: %0.2f" %cb_score)
print("The roc_auc score of CatBoost is: %0.2f" %cb_roc)

In [ ]:
from sklearn.metrics import classification_report
cr2 = classification_report(y_test, cb_pred)
print(cr2)

# Upvote if you like it or fork it. This helps us motivate to produce more notebooks for the community.